In [1]:
!apt install -y ffmpeg sox libsndfile1 git
!pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu118
!pip install git+https://github.com/m-bain/whisperx.git
!pip install gradio pydub ffmpeg-python faster-whisper pysubs2 tqdm wget
!pip install nemo_toolkit[asr] llama-index langchain demucs

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libsndfile1 is already the newest version (1.0.31-2build1).
git is already the newest version (1:2.34.1-1ubuntu1.10).
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
The following additional packages will be installed:
  libopencore-amrnb0 libopencore-amrwb0 libsox-fmt-alsa libsox-fmt-base libsox3 libwavpack1
Suggested packages:
  libsox-fmt-all
The following NEW packages will be installed:
  libopencore-amrnb0 libopencore-amrwb0 libsox-fmt-alsa libsox-fmt-base libsox3 libwavpack1 sox
0 upgraded, 7 newly installed, 0 to remove and 18 not upgraded.
Need to get 617 kB of archives.
After this operation, 1,764 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libopencore-amrnb0 amd64 0.1.5-1 [94.8 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libopencore-amrwb0 amd64 0.1.5-1 [49.1 kB]
Get:3 http://archive.ubu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.3/20.3 MB 82.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.9/92.9 kB 13.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.2/299.2 kB 30.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.7/75.7 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 kB 10.4 MB/s eta 0:00:00
  Created wheel for wget:

In [2]:
import demucs.separate
from pydub import AudioSegment
import ffmpeg
import os
import shlex
import shutil

sample_rate = 16000

def preprocess_audio(file_path, target_dBFS=-5, vocals_flg=True):
    file_dir = os.path.dirname(file_path)
    temp_outputs = os.path.join(file_dir, "temp_outputs")
    file_name, _ = os.path.splitext(file_path)
    tmp_file = file_path
    if vocals_flg:
        demucs.separate.main(shlex.split(f'-n htdemucs --two-stems=vocals "{file_path}" -o "{temp_outputs}"'))
        tmp_file = os.path.join(temp_outputs, "htdemucs", os.path.basename(file_name), "vocals.wav")
    output_file = f"{file_name}.16k.wav"
    ffmpeg.input(tmp_file).output(output_file, acodec='pcm_s16le', ar=sample_rate, ac=1).overwrite_output().run()
    if os.path.exists(temp_outputs):
        shutil.rmtree(temp_outputs)

    audio = AudioSegment.from_wav(output_file)
    if int(audio.dBFS) != target_dBFS:
        audio = audio.apply_gain(target_dBFS - audio.dBFS)
        audio.export(output_file, format="wav")
    return output_file

def get_dBFS(file_path):
    audio = AudioSegment.from_wav(file_path)
    return int(audio.dBFS)

In [3]:
from faster_whisper import WhisperModel
import whisperx
import torch
import os
import pysubs2
from tqdm import tqdm
from time import sleep

def transcribe_to_srt(audio_file, whisper_model="medium", compute_type="float16", beam_size=1, min_silence_duration_ms=500, initial_prompt="", is_align=True):
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model = WhisperModel(whisper_model, device=device, compute_type=compute_type)
    segments, info = model.transcribe(audio_file, beam_size=beam_size, word_timestamps=False,
                                  vad_filter=True, vad_parameters=dict(min_silence_duration_ms=min_silence_duration_ms),
                                  initial_prompt=initial_prompt)
    whisper_results = []
    total_duration = info.duration
    duration = 0
    with tqdm(total=total_duration, unit=" seconds") as pbar:
        for segment in segments:
            whisper_results.append(segment._asdict())
            segment_duration = segment.end - segment.start
            duration += segment_duration
            pbar.update(segment_duration)
        sleep(0.1)
        pbar.update(total_duration-duration)
    del model
    torch.cuda.empty_cache()
    if is_align:
        alignment_model, metadata = whisperx.load_align_model(language_code=info.language, device=device)
        result_aligned = whisperx.align(whisper_results, alignment_model, metadata, audio_file, device)
        whisper_results = result_aligned["segments"]
        del alignment_model
        torch.cuda.empty_cache()
    subs = pysubs2.load_from_whisper(whisper_results)
    text = subs.to_string(format_="srt")
    file_name, _ = os.path.splitext(audio_file)
    srt = f"{file_name}.{info.language}.srt"
    subs.save(srt)
    return text, srt


/usr/local/lib/python3.10/dist-packages/pyannote/audio/core/io.py:43: UserWarning: torchaudio._backend.set_audio_backend has been deprecated. With dispatcher enabled, this function is no-op. You can remove the function call.
  torchaudio.set_audio_backend("soundfile")
/usr/local/lib/python3.10/dist-packages/torch_audiomentations/utils/io.py:27: UserWarning: torchaudio._backend.set_audio_backend has been deprecated. With dispatcher enabled, this function is no-op. You can remove the function call.
  torchaudio.set_audio_backend("soundfile")


In [4]:
import os
import json
import wget
from omegaconf import OmegaConf
from nemo.collections.asr.models.msdd_models import NeuralDiarizer
import torch
import os
import shutil

def diarize_to_rttm(audio_file, config_type):
    device = "cuda" if torch.cuda.is_available() else "cpu"
    file_dir = os.path.dirname(audio_file)
    file_path_no_ext, _ = os.path.splitext(audio_file)
    file_name = os.path.basename(file_path_no_ext)
    work_home = os.path.join(file_dir, "nemo")
    if not os.path.exists(work_home):
        os.makedirs(work_home)

    meta = {
        'audio_filepath': f'{audio_file}',
        'offset': 0,
        'duration':  None,
        'label': "infer",
        'text': "-",
        'num_speakers': None,
        'rttm_filepath': None,
        'uniq_id': ""
    }
    manifest = os.path.join(work_home, 'manifest.json')
    if not os.path.exists(manifest):
        with open(manifest, 'w') as f:
            f.write(json.dumps(meta))
    mode_config = os.path.join(work_home, f'diar_infer_{config_type}.yaml')
    if not os.path.exists(mode_config):
        config_url = f'https://raw.githubusercontent.com/NVIDIA/NeMo/main/examples/speaker_tasks/diarization/conf/inference/diar_infer_{config_type}.yaml'
        mode_config = wget.download(config_url, work_home)

    config = OmegaConf.load(mode_config)
    config.num_workers = 1
    config.diarizer.manifest_filepath = manifest
    config.diarizer.out_dir = os.path.join(work_home, 'diarized')
    config.diarizer.vad.model_path = 'vad_multilingual_marblenet'
    config.diarizer.vad.parameters.onset = 0.8
    config.diarizer.vad.parameters.offset = 0.6
    config.diarizer.vad.parameters.pad_offset = -0.05
    config.diarizer.speaker_embeddings.model_path = 'titanet_large'
    config.diarizer.msdd_model.model_path = 'diar_msdd_telephonic'
    config.diarizer.oracle_vad = False
    config.diarizer.clustering.parameters.oracle_num_speakers=False
    model = NeuralDiarizer(cfg=config).to(device)
    model.diarize()
    del model
    torch.cuda.empty_cache()

    source_file = os.path.join(work_home, f"diarized/pred_rttms/{file_name}.rttm")
    if os.path.exists(source_file):
        shutil.move(source_file, file_dir)
        shutil.rmtree(work_home)

    rttm = os.path.join(file_dir, f"{file_name}.rttm")
    with open(rttm, 'r') as file:
        text = file.read()
    return text, rttm


In [5]:
import pysubs2

def get_speakers(rttm):
    speaker_ts = []
    speakers = set()
    with open(rttm, 'r') as f:
        lines = f.readlines()
        for line in lines:
            line_list = line.split(' ')
            s = int(float(line_list[5]) * 1000)
            e = s + int(float(line_list[8]) * 1000)
            speakers.add(line_list[11])
            speaker_ts.append([s, e, line_list[11]])
    return speaker_ts, speakers

def calculate_overlap_percentage(a_start, a_end, b_start, b_end):
    overlap_start = max(a_start, b_start)
    overlap_end = min(a_end, b_end)

    if overlap_start <= overlap_end:
        overlap_duration = overlap_end - overlap_start
        a_duration = a_end - a_start
        b_duration = b_end - b_start
        overlap_percentage = max(overlap_duration / min(a_duration, b_duration), overlap_duration / max(a_duration, b_duration))
        return overlap_percentage

    return 0.0

def find_max_overlap_or_closest(start, end, array, getter=lambda item: (item[0], item[1])):
    max_overlap_percentage = 0.0
    max_overlap_element = None
    closest_element = None
    closest_distance = float('inf')
    for item in array:
        a_start, a_end = getter(item)
        overlap_percentage = calculate_overlap_percentage(a_start, a_end, start, end)
        if overlap_percentage > max_overlap_percentage:
            max_overlap_percentage = overlap_percentage
            max_overlap_element = item
        b_midpoint = (start + end) / 2
        a_midpoint = (a_start + a_end) / 2
        distance = abs(b_midpoint - a_midpoint)
        if distance < closest_distance:
            closest_distance = distance
            closest_element = item
    return max_overlap_percentage, closest_element if max_overlap_percentage == 0.0 else max_overlap_element

def merge_speakers_sub(subs_file, speaker_ts, names_dict):
    subs = pysubs2.load(subs_file, encoding="utf-8")
    current_speaker = None
    speakers = {}
    draft = []
    warnings = []
    for line in subs:
        ws, we, wrd = line.start, line.end, line.text
        max_overlap_percentage, element = find_max_overlap_or_closest(ws, we, speaker_ts)
        speaker_code = element[2]
        if max_overlap_percentage < 0.5:
            speaker_name = names_dict.get(speaker_code, speaker_code)
            warnings.append(f"[%.2fs -> %.2fs] %s (%.2f %%):\"%s\" \n" % (ws/1000, we/1000, speaker_name, max_overlap_percentage * 100, wrd))
        speakers.setdefault(speaker_code, []).append(wrd)
        if current_speaker is not None and current_speaker != speaker_code:
            speaker_name = names_dict.get(current_speaker, current_speaker)
            draft.append(f"{speaker_name}: \"{' '.join(speakers[current_speaker])}\" \n")
            speakers[current_speaker] = []
        current_speaker = speaker_code

    if current_speaker is not None:
        speaker_name = names_dict.get(current_speaker, current_speaker)
        draft.append(f"{speaker_name}: \"{' '.join(speakers[current_speaker])}\" \n")

    return ''.join(draft), ''.join(warnings)

In [6]:
import shutil
import os
from llama_index import SimpleDirectoryReader, VectorStoreIndex, ServiceContext, StorageContext, load_index_from_storage
from llama_index.node_parser import SimpleNodeParser
from langchain.text_splitter import RecursiveCharacterTextSplitter
from llama_index.llms import AzureOpenAI
from llama_index.embeddings.openai import OpenAIEmbedding

def engine_building(input_file,api_base,api_key,api_version,engine,embed_model_name,embed_deployment_name,embed_api_version):
    llm = AzureOpenAI(
        engine=engine,
        model="gpt-35-turbo-16k",
        temperature=0.3,
        max_tokens=4096,
        api_base=api_base,
        api_key=api_key,
        api_type="azure",
        api_version=api_version,
    )
    embed_model = OpenAIEmbedding(
        model_name=embed_model_name,
        deployment_name=embed_deployment_name,
        api_key=api_key,
        api_base=api_base,
        api_type="azure",
        api_version=embed_api_version,
    )
    service_context = ServiceContext.from_defaults(llm=llm,embed_model=embed_model)
    documents = SimpleDirectoryReader(input_files=[input_file]).load_data()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
    parser = SimpleNodeParser.from_defaults(text_splitter=text_splitter)
    nodes = parser.get_nodes_from_documents(documents)
    index = VectorStoreIndex(nodes=nodes,service_context=service_context)
    folder_name = "storage"
    shutil.make_archive(folder_name, "zip", folder_name)
    return index.as_query_engine(similarity_top_k=5), f"{folder_name}.zip"

def engine_loading(input_file,api_base,api_key,api_version,engine,embed_model_name,embed_deployment_name,embed_api_version):
    file_dir = os.path.dirname(input_file)
    folder = os.path.join(file_dir, "storage")
    shutil.unpack_archive(input_file, folder, "zip")
    llm = AzureOpenAI(
        engine=engine,
        model="gpt-35-turbo-16k",
        temperature=0.3,
        max_tokens=4096,
        api_base=api_base,
        api_key=api_key,
        api_type="azure",
        api_version=api_version,
    )
    embed_model = OpenAIEmbedding(
        model_name=embed_model_name,
        deployment_name=embed_deployment_name,
        api_key=api_key,
        api_base=api_base,
        api_type="azure",
        api_version=embed_api_version,
    )
    service_context = ServiceContext.from_defaults(llm=llm,embed_model=embed_model)
    storage_context = StorageContext.from_defaults(persist_dir=folder)
    index = load_index_from_storage(storage_context, service_context=service_context)
    return index.as_query_engine(similarity_top_k=10)

In [8]:
import gradio as gr

def preprocess(file_path, target_dBFS, vocals_flg):
    output_file = preprocess_audio(file_path, target_dBFS, vocals_flg)
    return output_file, output_file

def transcribe(audio_file, whisper_model, compute_type, beam_size, min_silence_duration_ms, initial_prompt, is_align):
    text, srt = transcribe_to_srt(audio_file, whisper_model, compute_type, beam_size, min_silence_duration_ms, initial_prompt, is_align)
    return text, srt, srt

def diarize(audio_file, config_type):
    text, rttm_file = diarize_to_rttm(audio_file, config_type)
    return text, rttm_file, rttm_file

def show_speakers(rttm_file):
    _, speakers = get_speakers(rttm_file.name)
    speakers_list = list(speakers)
    return { "headers":speakers_list, "data":[speakers_list] }

def meger_text(rttm_file, subs_file, speakers):
    speaker_ts, _ = get_speakers(rttm_file.name)
    content, warning = merge_speakers_sub(subs_file.name, speaker_ts, speakers.to_dict('records')[0])
    return gr.Dropdown(["all"] + speakers.iloc[0].values.tolist()), content, warning, content

def speaker_filter(speakers_selector, cached_text):
    parts = cached_text.split('\n')
    filtered_text = [part for part in parts if part.startswith(speakers_selector + ':')]
    return '\n'.join(filtered_text)

def save_file(text):
    file = "output.txt"
    with open(file, 'w') as f:
        f.write(text)
    return file, file

def generate(input_file,api_base,api_key,api_version,engine,embed_model_name,embed_deployment_name,embed_model_api_version):
    llm_engine, zip = engine_building(input_file.name,api_base,api_key,api_version,engine,embed_model_name,embed_deployment_name,embed_model_api_version)
    return zip, gr.Button(visible=False), gr.Tabs.update(selected=1),llm_engine

def load_datas(input_datas_file,api_base,api_key,api_version,engine,embed_model_name,embed_deployment_name,embed_model_api_version):
    llm_engine = engine_loading(input_datas_file.name,api_base,api_key,api_version,engine,embed_model_name,embed_deployment_name,embed_model_api_version)
    return llm_engine

def user(message, history):
    return "", history + [[message, None]]

def bot(history,llm_engine):
    user_message = history[-1][0]
    response = llm_engine.query(user_message)
    history.append([None, str(response)])
    return history

def send_to_other_tab(info, target_tab):
    return info, gr.Tabs.update(selected=target_tab)

with gr.Blocks() as demo:
    gr.Markdown("# 没想好名字的AI工具")
    with gr.Tabs() as tabs:
        with gr.TabItem("预处理", id=0):
            gr.Markdown("将音频或视频转换为采样速率16k的wav文件，可进行人声分离及说话分贝值调整")
            with gr.Row():
                with gr.Column():
                    raw_audio_input = gr.Audio(label="Input Audio", type="filepath")
                    decibel = gr.Slider(-50, 0, step=1, label = "分贝", info="声音过轻可在此处调节")
                    vocals_flg = gr.Checkbox(value=True, label="人声分离", info="去除背景音")
                    preprocess_audio_btn = gr.Button("预处理")
                with gr.Column():
                    cached_preprocess = gr.State()
                    pre_audio_output = gr.Audio(label="Output Audio", type="filepath")
                    with gr.Row():
                        pre_to_transcription_btn = gr.Button("发送到语音转录")
                        pre_to_speaker_recognition_btn = gr.Button("发送到说话人识别")
                raw_audio_input.upload(get_dBFS, inputs=raw_audio_input, outputs=decibel)
                preprocess_audio_btn.click(preprocess, inputs=[raw_audio_input,decibel,vocals_flg], outputs=[pre_audio_output,cached_preprocess])
        with gr.TabItem("语音转录", id=1):
            gr.Markdown("音频文件转录成文字")
            with gr.Row():
                with gr.Column():
                    wav_audio_input = gr.Audio(label="Input Audio",info="推荐使用采样速率16K的音频文件", type="filepath")
                    whisper_models = gr.Dropdown(["medium", "large-v2"], value="medium", label="Models", info="选择转录模型")
                    compute_type = gr.Radio(["float16", "float32"], value="float16", label="compute_type", info="单精度或双精度")
                    beam_size = gr.Slider(1, 10, step=1, value=5, label = "beam_size")
                    vad_parameters = gr.Slider(100, 10000, step=100, value=2000, label = "vad_min_silence_duration_ms")
                    initial_prompt = gr.Textbox(label="initial_prompt")
                    is_align_flg = gr.Checkbox(label="对齐", info="wav2vec2模型")
                    transcribe_btn = gr.Button("转录")
                with gr.Column():
                    cached_srt = gr.State()
                    subs_preview = gr.Textbox(label="字幕预览", show_copy_button=True)
                    subs_file = gr.File(label="字幕文件",file_types=['.str','.ass'])
                    with gr.Row():
                        send_srt_to_merge_btn = gr.Button("发送到合并信息")
                        send_srt_to_llm_btn = gr.Button("发送到LLM知识库")
                transcribe_btn.click(transcribe, inputs=[wav_audio_input,whisper_models,compute_type, beam_size,vad_parameters,initial_prompt,is_align_flg], outputs=[subs_preview, subs_file, cached_srt])
        with gr.TabItem("说话人分类", id=2):
            gr.Markdown("在多人会话中，将不同说话人进行分类")
            with gr.Row():
                with gr.Column():
                    source_audio_input = gr.Audio(label="Input Audio", type="filepath")
                    config_type = gr.Dropdown(["general", "meeting", "telephonic"], value="telephonic", label="配置类型", info="预配置模版")
                    diarize_btn = gr.Button("分类")
                with gr.Column():
                    cached_rttm = gr.State()
                    rttm_preview = gr.Textbox(label="说话人分类预览", show_copy_button=True)
                    rttm_file = gr.File(label="rttm文件", file_types=['.rrtm'])
                    send_rttm_to_merge_btn = gr.Button("发送到合并信息")
            diarize_btn.click(diarize, inputs=[source_audio_input,config_type], outputs=[rttm_preview,rttm_file,cached_rttm])
        with gr.TabItem("合并信息", id=3):
            gr.Markdown("将说话人和字幕文件进行匹配")
            with gr.Row():
                with gr.Column():
                    rttm_file_input = gr.File(label="rttm文件", file_types=['.rttm'])
                    subs_file_input = gr.File(label="字幕文件",file_types=['.srt'])
                    speakers = gr.Dataframe(label="说话人列表",row_count=(1, "fixed"))
                    merge_btn = gr.Button("合并")
                with gr.Column():
                    speakers_selector = gr.Dropdown(["all"], value="all", label="说话人选择")
                    text_preview = gr.Textbox(label="合成预览", show_copy_button=True)
                    warning_show = gr.Textbox(label="匹配度警告")
                    cached_text = gr.State()
                    make_file_btn = gr.Button("生成文件")
                with gr.Column():
                    cached_output = gr.State()
                    text_file = gr.File(label="合并文件", file_types=['text'])
                    send_output_to_llm_btn = gr.Button("发送到LLM知识库")
                rttm_file_input.upload(show_speakers, inputs=rttm_file_input, outputs=speakers)
                merge_btn.click(meger_text, inputs=[rttm_file_input,subs_file_input,speakers], outputs=[speakers_selector, text_preview, warning_show, cached_text])
                speakers_selector.select(speaker_filter, inputs=[speakers_selector, cached_text], outputs=text_preview)
                make_file_btn.click(save_file, inputs=cached_text, outputs=[text_file,cached_output])
        with gr.TabItem("LLM知识库", id=4):
            gr.Markdown("你可以直接和你的上传文件进行对话")
            with gr.Row():
                with gr.Column(scale=1):
                    llm_selector = gr.Dropdown(["Azure OpenAI"], value="Azure OpenAI", label="LLM选择")
                    api_base = gr.Textbox(label="api_base")
                    api_key = gr.Textbox(label="api_key", type="password")
                    api_version = gr.Textbox(label="api_version")
                    engine = gr.Textbox(label="engine")
                    embed_model_name = gr.Textbox(label="embed_model_name", value="text-embedding-ada-002")
                    embed_deployment_name = gr.Textbox(label="embed_deployment_name")
                    embed_model_api_version = gr.Textbox(label="embed_model_api_version", value="2023-05-15")
                    with gr.Tabs() as files:
                        with gr.TabItem("未处理文件", id=0):
                            input_file = gr.File(label="私人文件", file_types=['text','.srt'])
                            generation_btn = gr.Button("生成知识库")
                        with gr.TabItem("知识库文件", id=1):
                            input_datas_file = gr.File(label="知识库文件", file_types=['.zip'])
                            load_btn = gr.Button("加载")
                with gr.Column(scale=3):
                    llm_engine = gr.State()
                    llm_index = gr.State()
                    chatbot = gr.Chatbot()
                    msg = gr.Textbox(label="Input message")
                    clear = gr.Button("Clear")
                generation_btn.click(generate, inputs=[input_file,api_base,api_key,api_version,engine,embed_model_name,embed_deployment_name,embed_model_api_version], outputs=[input_datas_file,load_btn,files, llm_engine])
                load_btn.click(load_datas,inputs=[input_datas_file,api_base,api_key,api_version,engine,embed_model_name,embed_deployment_name,embed_model_api_version], outputs=llm_engine)
                msg.submit(user, [msg, chatbot], [msg, chatbot], queue=False).then(bot, [chatbot, llm_engine], chatbot)
                clear.click(lambda: None, None, chatbot, queue=False)
    pre_to_transcription_btn.click(send_to_other_tab, inputs=[cached_preprocess, gr.State(value=1)], outputs=[wav_audio_input,tabs])
    pre_to_speaker_recognition_btn.click(send_to_other_tab, inputs=[cached_preprocess, gr.State(value=2)], outputs=[source_audio_input,tabs])
    send_srt_to_merge_btn.click(send_to_other_tab, inputs=[cached_srt, gr.State(value=3)], outputs=[subs_file_input,tabs])
    send_srt_to_llm_btn.click(send_to_other_tab, inputs=[cached_srt, gr.State(value=4)], outputs=[input_file,tabs])
    send_rttm_to_merge_btn.click(send_to_other_tab, inputs=[cached_rttm, gr.State(value=3)], outputs=[rttm_file_input,tabs])
    send_output_to_llm_btn.click(send_to_other_tab, inputs=[cached_output, gr.State(value=4)], outputs=[input_file,tabs])
demo.queue(max_size=50).launch(debug=True, share=True, inline=False)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://17dbb92d991ee9a1ba.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://17dbb92d991ee9a1ba.gradio.live
